In [1]:
# Check python version and working dir
! python -V
! ls .

Python 2.7.17 :: Anaconda, Inc.
Programs API.ipynb          upload_from_csv.pyc
Vericast-reports-tool.ipynb uploading_epg_csv.ipynb
__init__.py                 vericast_reports.py
upload_from_csv.py


In [2]:
import os
import json
import pandas as pd
import datetime
import dateutil.parser
         

# Change directory to filesdir
os.chdir('../../data/')
class VericastMatchReporter:
    """
    @params: filename, start_date, start_time, end_date, end_time, time_zone
    """
    filename = None
    channel = None
    report_name = 'vericast-api-matches'
    report_file_extension = 'xslx'
    df = pd.DataFrame(columns=['title','length','album','artist','start_time_utc'])
    matches_between_dates = None
    start_time = None
    end_time = None
    dfeng = None
    
    
    def __init__(self, **kwargs):
        """Initilize report, make a pandas dataframe with the matches."""
        
        self.filename = kwargs['filename']
        self.channel = self.filename.split('/')[-1].split('-')[-1].split('.')[0]
        self.report_name += '-' + self.channel 
        try:
            # Make a df with engineer's times and bring them to UTC for correct comparison
            start_time_naive = dateutil.parser.parse(kwargs['start_date']+'T'+kwargs['start_time'])
            end_time_naive = dateutil.parser.parse(kwargs['end_date']+'T'+kwargs['end_time'])
            dfeng = pd.DataFrame({'naive_datetime':[start_time_naive,end_time_naive]}, index=['filter_start','filter_end'])
            dfeng['local_datetime'] = pd.DatetimeIndex(dfeng['naive_datetime']).tz_localize(tz =kwargs['time_zone'])
            dfeng['utc_datetime'] = pd.DatetimeIndex(dfeng['local_datetime']).tz_convert(tz ='UTC')
            self.dfeng = dfeng
            self.report_name +=' {} to {}.'.format(self.dfeng.loc['filter_start']['local_datetime'],self.dfeng.loc['filter_end']['local_datetime'])
            self.report_name =os.getcwd() + '.{}'.format(self.report_file_extension)
        except:
            raise
        with open(self.filename) as f:
            for line in f.readlines():
                # Append line into df
                self.df = self.df.append(json.loads(line), ignore_index=True)
                
        # Cast dates as aware datetime with timezone 'UTC'
        self.df['start_time_utc'] = pd.to_datetime(self.df['start_time_utc'], yearfirst=True, utc=True)
        print('Reporter initialized.')
        
    def create_report_between_times_xlsx(self):
        # Make a mask boolean mask between dates
        mask = (self.df['start_time_utc'] >= self.dfeng.loc['filter_start']['utc_datetime']) & (self.df['start_time_utc'] <= self.dfeng.loc['filter_end']['utc_datetime'])
        self.matches_between_dates = self.df.loc[mask]
        # Write report to xslx
        writer = pd.ExcelWriter(self.report_name)
        matches_between_dates.to_excel(writer, sheet_name=self.channel, index=False)
        writer.save()
        writer.close()
        print('File {} with {} matches has been created.'.format(self.report_name, self.matches_between_dates.count()))
        return self.matches_between_dates
    
    def __str__(self):
        return 'Vericast API matches from '+self.channel.replace('_', ' ').upper()+' between times '+str(self.start_time)+' and '+str(self.end_time)+' UTC.'

    def __repr__(self):
        return "<{}: Channel {}>".format(self.__class__.__name__, self.channel)

In [4]:
# Create a reporter instance with engineer's data
report = VericastMatchReporter(
    filename='matches-rte.json', 
    start_date='2018-01-29', 
    start_time='12:00', 
    end_date='2018-02-04',
    end_time='20:13',
    time_zone='america/new_york'
)

Reporter initialized.


In [5]:
# show the df created by reporter from file
sample = report.df.sample(100).sort_values(by=['start_time_utc'])
sample

,title,length,album,artist,start_time_utc
771,Texas And Beyond,29,Classic Film & TV 2: Matinee Mania,Johnny Scott,2018-01-25 10:31:28+00:00
763,With You,40,Chart Pop 2,Cody William Falkosky,2018-01-25 13:46:30+00:00
747,Getting In The Mood,9,Chart Songs,"Anders Paul Niska, Jonas Axel Andreas Edquist,...",2018-01-25 14:43:42+00:00
725,Upper Cut,7,Essential Underscores,Gavin Griffiths,2018-01-26 04:27:16+00:00
723,Sports Tech,3,Sports Scored,Bud Guin,2018-01-26 04:27:59+00:00
721,Rush,4,Movie Mania,"Marty Irwin, Raj Kamahl",2018-01-26 04:29:48+00:00
706,My Own World,11,The Heat: Summer Dance Vibes,Robert William Burn,2018-01-26 13:39:17+00:00
698,Puffin' Billy,29,Busy Housewives,Edward White,2018-01-26 15:56:50+00:00
697,Carmen -Suite No2 (Habanera),26,Bizet G.,Georges Bizet,2018-01-26 16:27:16+00:00
696,Busy Bees,18,Pizzicato Strings,"Bob Bradley, Matt Sanchez, Steve Dymond",2018-01-26 16:59:07+00:00


In [6]:
# Make a mask boolean mask between dates
mask = (sample['start_time_utc'] >= report.dfeng.loc['filter_start']['utc_datetime']) & (sample['start_time_utc'] <= report.dfeng.loc['filter_end']['utc_datetime'])
matches_between_dates = sample.loc[mask]

In [7]:
dirw= os.getcwd()
writer = pd.ExcelWriter(dirw+'my_file32.xlsx')
matches_between_dates.to_excel(writer, sheet_name='Whatever', index=False)
writer.save()
writer.close()

In [8]:
# show the dfeng created by reporter from engineer's data
report.dfeng.head(2)

,naive_datetime,local_datetime,utc_datetime
filter_start,2018-01-29 12:00:00,2018-01-29 12:00:00-05:00,2018-01-29 17:00:00+00:00
filter_end,2018-02-04 20:13:00,2018-02-04 20:13:00-05:00,2018-02-05 01:13:00+00:00


In [19]:
# Numer of rows that match
len(matches_between_dates.length)

22